In [1]:
import awswrangler as wr
import pandas as pd
pd.set_option('display.max_rows', 300)
import pyarrow as pa
import functools
import boto3
import plotly.express as ex
import plotly.graph_objects as go


In [2]:
my_filter = lambda x: x["CoType"] == "I"

wr.config.s3_endpoint_url = "http://192.168.1.4:8333"
bucket = "tmp_part"


In [3]:
# wr.s3.read_parquet_metadata(f"s3://{bucket}/", dtype={
#     "siNETTDB" : pa.int32()
# })


In [4]:
my_filter = lambda x: x["CoType"] == "I"

wr.config.s3_endpoint_url = "http://192.168.1.4:8333"
bucket = "tmp_part"

partDf = wr.s3.read_parquet(f"s3://{bucket}/",  dataset=True, partition_filter=my_filter, columns=["CaseID", "Description", "fVAL", "DamageCond"])
partDf

,CaseID,Description,fVAL,DamageCond,CoType
0,12657878,Bonnet,550.00,Bent/dented,I
1,12657878,Grille Badge,48.90,Cut/chipped,I
2,12657878,Grille Front Moulding,NaN,Broken,I
3,12657878,Grille Mounting Bracket,34.72,Broken,I
4,12657878,Bumper Front Cover,350.00,Creased/torn,I
...,...,...,...,...,...
26637941,12908162,Sealant,80.00,,I
26637942,12908162,Windscreen Glass - Front,309.20,,I
26637943,12908163,Sealant,70.00,,I
26637944,12908163,Tint Film,280.00,,I


In [5]:
partDf["DamageCond"] = partDf["DamageCond"].str.lower()
partDf["Description"] = partDf["Description"].str.lower()
partDf

,CaseID,Description,fVAL,DamageCond,CoType
0,12657878,bonnet,550.00,bent/dented,I
1,12657878,grille badge,48.90,cut/chipped,I
2,12657878,grille front moulding,NaN,broken,I
3,12657878,grille mounting bracket,34.72,broken,I
4,12657878,bumper front cover,350.00,creased/torn,I
...,...,...,...,...,...
26637941,12908162,sealant,80.00,,I
26637942,12908162,windscreen glass - front,309.20,,I
26637943,12908163,sealant,70.00,,I
26637944,12908163,tint film,280.00,,I


In [6]:
rawPartsTotal = len(partDf)

In [7]:
costThreshold = 100

In [8]:
removedByLowCost = len(partDf[(partDf["fVAL"] < costThreshold)])


In [9]:
nullCostParts = len(partDf[partDf["fVAL"].isnull()]) / len(partDf)
nullCostParts


0.20404673843846668

In [10]:
DmgCondFilter = [
    "no visible damage", 
    "to check", 
    "useable",
    "not necessary",
    "not damaged",
    "slightly bent",
    "slightly cut",
    "slightly scratched",
    "slightly dented",
    "visually intact"
]

In [11]:
removedByCond = len(partDf[partDf["DamageCond"].isin(DmgCondFilter)])
removedByCond

1906222

In [12]:
removedByDash  = len(partDf[partDf["Description"] == "-"])


In [13]:
removedByNoCost  = len(partDf[partDf["fVAL"].isna()])


In [14]:
filterPartDf = partDf[(partDf["fVAL"] > costThreshold)]
filterPartDf = filterPartDf[~filterPartDf["DamageCond"].isin(DmgCondFilter)]
filterPartDf = filterPartDf[filterPartDf["Description"] != "-"]


In [15]:
totalRemovedPart = len(partDf) - len(filterPartDf)

In [16]:
import plotly.express as ex
import plotly.graph_objects as go


fig = go.Figure(data=[go.Pie(labels=["Part Data Used", 
                                     "Filtered Parts", \
                                     
                                     ], \
                             values=[len(filterPartDf), totalRemovedPart])])
fig.update_layout(title="Partlist Filtering")

fig.show()


In [17]:
# filteredPartDf["DamageCond"].value_counts().head(n=100).reset_index()
# filterPartDf["Description"].value_counts().reset_index().head(300)

In [18]:
filterPartDf["fVAL"].sum()

9173289332.17999

In [19]:
filterPartDf["fVAL"].sum() / partDf["fVAL"].sum()

0.9623767590279574

In [20]:

wr.config.s3_endpoint_url = "http://192.168.1.4:8333"
bucket2 = "tmp_file"

In [21]:
frontViewLeftDf = wr.s3.read_parquet(f"s3://{bucket2}/",  dataset=True, partition_filter= lambda x: x["StdDocDesc"] == "Front View Left", columns=["CaseID", "iDOCID", "FinalDate", "StdDocDesc"])
frontViewRightDf = wr.s3.read_parquet(f"s3://{bucket2}/",  dataset=True, partition_filter= lambda x: x["StdDocDesc"] == "Front View Right", columns=["CaseID", "iDOCID", "FinalDate", "StdDocDesc"])
rearViewRightDf = wr.s3.read_parquet(f"s3://{bucket2}/",  dataset=True, partition_filter= lambda x: x["StdDocDesc"] == "Rear View Right", columns=["CaseID", "iDOCID", "FinalDate", "StdDocDesc"])
rearViewLeftDf = wr.s3.read_parquet(f"s3://{bucket2}/",  dataset=True, partition_filter= lambda x: x["StdDocDesc"] == "Rear View Left", columns=["CaseID", "iDOCID", "FinalDate", "StdDocDesc"])
allViewFilesDf = [frontViewLeftDf, frontViewRightDf,  rearViewRightDf, rearViewLeftDf]

In [22]:
dfWithUniqueView = []
for dfId in range(len(allViewFilesDf)):
    df = allViewFilesDf[dfId]
    df["FinalDate"] = pd.to_datetime(df["FinalDate"])
    df.sort_values(by="FinalDate", inplace=True)
    df = df.groupby("CaseID").head(1)
    dfWithUniqueView.append(df)
fileDf = pd.concat(dfWithUniqueView)

In [23]:
validFileDf = fileDf.groupby("CaseID")["StdDocDesc"].apply(list).reset_index()
validFileDf["complete"] = validFileDf["StdDocDesc"].apply(lambda x : len(set(x)) == 4)
validFileDf = validFileDf[validFileDf["complete"] == True]
validFileDf

,CaseID,StdDocDesc,complete
0,10000004,"[Front View Left, Front View Right, Rear View ...",True
1,10000008,"[Front View Left, Front View Right, Rear View ...",True
2,10000012,"[Front View Left, Front View Right, Rear View ...",True
3,10000016,"[Front View Left, Front View Right, Rear View ...",True
4,10000021,"[Front View Left, Front View Right, Rear View ...",True
...,...,...,...
1000117,13564503,"[Front View Left, Front View Right, Rear View ...",True
1000118,13564507,"[Front View Left, Front View Right, Rear View ...",True
1000119,13564510,"[Front View Left, Front View Right, Rear View ...",True
1000120,13564513,"[Front View Left, Front View Right, Rear View ...",True


In [24]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"
bucket3 = "tmp_case"

In [25]:
# wr.s3.describe_objects(f"s3://{bucket3}/")

In [26]:
tempDf = wr.s3.read_parquet(f"s3://{bucket3}/" ,dataset=True, chunked=100)
for i in tempDf :
    print(sorted(i.columns.tolist()))
    break


['1st_Attended_By', 'Accident_Scene_Photos_Count', 'Adjuster', 'Adjuster_Co_ID', 'Adjuster_HQ_ID', 'Adjuster_Narratives', 'Adjuster_Phone_Number', 'Adjuster_Rpt_Doc_Exist', 'Adjuster_Rpt_Submit_Datetime', 'Agency_Name', 'Agent_Address', 'Agent_PIAM_Code', 'Amount_Exclude_GST', 'Assembly_Type', 'Authorization_Datetime', 'Brake_System_Footbrake', 'CDriver_Drv_License_No', 'CDriver_Email', 'CDriver_ID1', 'CDriver_ID2', 'CDriver_Name', 'CDriver_Nationality', 'Cancel_Remarks', 'Car_Age', 'CaseID', 'Case_Remarks', 'Chassis_No', 'Circumstances_of_Accident', 'Circumstances_of_Accident_Code', 'Claim_Number', 'Claim_Reg_Datetime', 'Claim_Type', 'Claim_Type_Mask', 'Cmt_Address_1', 'Cmt_Address_2', 'Cmt_Birth_Date', 'Cmt_City', 'Cmt_Country', 'Cmt_Drv_License_No', 'Cmt_Email', 'Cmt_Gender', 'Cmt_ID1', 'Cmt_ID2', 'Cmt_Mobile_Phone_No', 'Cmt_Name', 'Cmt_Nationality', 'Cmt_Office_Phone_No', 'Cmt_Other_Phone_No', 'Cmt_Postcode', 'Cmt_State', 'Collision_With', 'Colour_of_Vehicle', 'Condition_of_Damage'

In [27]:
# tempdf = pd.read_parquet("/home/alextay96/Desktop/new_workspace/partlist_prediction/data/raw/complete_encoded_no_limit.parquet")
# tempdf["Vehicle_Type"].value_counts().reset_index().head(10)


In [28]:
myFilter = lambda x : x["Vehicle_Type"] == "Hatchback - 5 Dr" or x["Vehicle_Type"] == "Saloon - 4 Dr" or x["Vehicle_Type"] == "MPV" \
or x["Vehicle_Type"] == "SUV - 5 Dr" or x["Vehicle_Type"] == "Pickup - 4 Dr Dbl.Cab" 
targetCols = ["CaseID", 
"Vehicle_Class", 
"Vehicle_Type", 
"Gross_Offer_Insurer", 
"Circumstances_of_Accident", 
"Claim_Type", "Model",
        "Assembly_Type",
        "Vehicle_Still_Driveable",
        "NCB_Stat",
        "Assembly_Type",
        "Sum_Insured",
        "Repairer",
        "Repairer_Apprv_Count",
        "Collision_With",
        "Handling_Insurer", ]

# targetCols = list()
caseDf = wr.s3.read_parquet(f"s3://{bucket3}/" ,partition_filter=myFilter,dataset=True, columns=targetCols)
caseDf

,CaseID,Vehicle_Class,Gross_Offer_Insurer,Circumstances_of_Accident,Claim_Type,Model,Assembly_Type,Vehicle_Still_Driveable,NCB_Stat,Sum_Insured,Repairer,Repairer_Apprv_Count,Collision_With,Handling_Insurer,Vehicle_Type
0,10407851,2,600.00,Breakage of Windscreen,WS,Myvi 2011 On,3,<NA>,0,600.0,Loke Yew Auto Windscreen Specialist (HQ),0,,Takaful Ikhlas General Berhad,Hatchback - 5 Dr
1,10407853,2,468.20,,WS,Kelisa 2001 On,3,<NA>,0,500.0,DR Cermin Sdn Bhd (HQ),0,,Etiqa General Takaful Berhad,Hatchback - 5 Dr
2,10407855,2,400.00,Breakage of Windscreen,WS,Myvi 2005 On,3,<NA>,0,400.0,"Koon Lee Auto Glass Sdn Bhd (Bukit Tinggi,Klang)",0,,Takaful Ikhlas General Berhad,Hatchback - 5 Dr
3,10407856,<NA>,0.00,,TP,,<NA>,<NA>,1,NaN,Maju Mesra Spraying Services (HQ),81,Private Vehicle,AmGeneral Insurance Berhad,Hatchback - 5 Dr
4,10407858,2,1516.82,Collided into motorcyclist,OD,Myvi 2011 On,3,0,1,28000.0,GI Hoe Motor Works Sdn Bhd (HQ),2633,Private Vehicle,Zurich General Insurance Malaysia Berhad,Hatchback - 5 Dr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450277,10150978,2,17800.00,,OD KFK,Sentra 1995 On,3,0,0,21000.0,Pooh Hui Paint Spraying Workshop Sdn Bhd (HQ),2838,Private Vehicle,Syarikat Takaful Malaysia Am Berhad,Saloon - 4 Dr
2450278,10150980,2,559.00,Breakage of Windscreen,WS,Saga 2008 On,3,<NA>,0,600.0,Nanmar Motor Service Sdn Bhd (HQ),0,,TUNE PROTECT MALAYSIA,Saloon - 4 Dr
2450279,10150981,2,9061.74,Collided into motorcyclist,OD,City 2014 On,3,1,1,46000.0,Actmar Sdn Bhd (HQ),1929,Private Vehicle,AIG Malaysia Insurance Berhad,Saloon - 4 Dr
2450280,10150996,2,4701.85,Collision- Head to Rear (Insured Hit TP),OD,Persona 2016 On,3,0,1,50000.0,Lon G Auto Services Sdn Bhd (Lon G),5794,Private Vehicle,Zurich General Takaful Malaysia Berhad,Saloon - 4 Dr


In [29]:
caseDf

,CaseID,Vehicle_Class,Gross_Offer_Insurer,Circumstances_of_Accident,Claim_Type,Model,Assembly_Type,Vehicle_Still_Driveable,NCB_Stat,Sum_Insured,Repairer,Repairer_Apprv_Count,Collision_With,Handling_Insurer,Vehicle_Type
0,10407851,2,600.00,Breakage of Windscreen,WS,Myvi 2011 On,3,<NA>,0,600.0,Loke Yew Auto Windscreen Specialist (HQ),0,,Takaful Ikhlas General Berhad,Hatchback - 5 Dr
1,10407853,2,468.20,,WS,Kelisa 2001 On,3,<NA>,0,500.0,DR Cermin Sdn Bhd (HQ),0,,Etiqa General Takaful Berhad,Hatchback - 5 Dr
2,10407855,2,400.00,Breakage of Windscreen,WS,Myvi 2005 On,3,<NA>,0,400.0,"Koon Lee Auto Glass Sdn Bhd (Bukit Tinggi,Klang)",0,,Takaful Ikhlas General Berhad,Hatchback - 5 Dr
3,10407856,<NA>,0.00,,TP,,<NA>,<NA>,1,NaN,Maju Mesra Spraying Services (HQ),81,Private Vehicle,AmGeneral Insurance Berhad,Hatchback - 5 Dr
4,10407858,2,1516.82,Collided into motorcyclist,OD,Myvi 2011 On,3,0,1,28000.0,GI Hoe Motor Works Sdn Bhd (HQ),2633,Private Vehicle,Zurich General Insurance Malaysia Berhad,Hatchback - 5 Dr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450277,10150978,2,17800.00,,OD KFK,Sentra 1995 On,3,0,0,21000.0,Pooh Hui Paint Spraying Workshop Sdn Bhd (HQ),2838,Private Vehicle,Syarikat Takaful Malaysia Am Berhad,Saloon - 4 Dr
2450278,10150980,2,559.00,Breakage of Windscreen,WS,Saga 2008 On,3,<NA>,0,600.0,Nanmar Motor Service Sdn Bhd (HQ),0,,TUNE PROTECT MALAYSIA,Saloon - 4 Dr
2450279,10150981,2,9061.74,Collided into motorcyclist,OD,City 2014 On,3,1,1,46000.0,Actmar Sdn Bhd (HQ),1929,Private Vehicle,AIG Malaysia Insurance Berhad,Saloon - 4 Dr
2450280,10150996,2,4701.85,Collision- Head to Rear (Insured Hit TP),OD,Persona 2016 On,3,0,1,50000.0,Lon G Auto Services Sdn Bhd (Lon G),5794,Private Vehicle,Zurich General Takaful Malaysia Berhad,Saloon - 4 Dr


In [30]:
caseDf["Vehicle_Type"].value_counts().reset_index().head(10)


,index,Vehicle_Type
0,Saloon - 4 Dr,1043417
1,Hatchback - 5 Dr,685372
2,MPV,328378
3,SUV - 5 Dr,255113
4,Pickup - 4 Dr Dbl.Cab,138002


In [31]:
caseDf["Claim_Type"].value_counts().head(100)

WS        883446
OD        592622
TP        365581
OD KFK    241748
TP UL     168541
TP KFK    144586
OD EXW     15308
LU         13484
TP BI      11062
TF          5766
SC          2870
OD MNT      1888
OD TFR      1353
TP PD        926
OD TAC       814
TP SB        224
OD GRG        63
Name: Claim_Type, dtype: Int64

In [32]:
circumstancesToRemove = [
    "Breakage of Windscreen",
    "Stolen Whilst Parked",
    "Fire, explosion or lightning",
    "Malicious damage",
    "Flood",
    "Fell into a Ravine",
    "Stolen due to Violence",
    'Self Ignition- Unknown Cause',
    "Boarding or Alighting",
    "Damage due to Flood and Typhoon",
    "Theft of parts and/or accessories",
    "Stolen and recovered"
]

In [33]:
claimType = [
    "OD",
    "TP"
]

In [34]:
removeByClaimType = len(caseDf[~caseDf["Claim_Type"].str.contains("OD|TP")])
removeByClaimType

905566

In [35]:
removedByVehicleClass = len(caseDf[caseDf["Vehicle_Class"] != 2])
removedByVehicleClass

15906

In [36]:
removedByCircumstances = len(caseDf[caseDf["Circumstances_of_Accident"].isin(circumstancesToRemove)])
removedByCircumstances

517433

In [37]:
removedByGrossOfferInsurer = len(caseDf[caseDf["Gross_Offer_Insurer"] <= 0])
removedByGrossOfferInsurer

372439

In [38]:
removedByFiles = len(caseDf[~caseDf["CaseID"].isin(validFileDf["CaseID"].unique().tolist())])
removedByFiles

1615472

In [39]:
removedByParts = len(caseDf[~caseDf["CaseID"].isin(filterPartDf["CaseID"].unique().tolist())])
removedByParts

710680

In [40]:
# totalScope = len(caseDf) - removeByClaimType


In [41]:
targetCaseDf = caseDf[caseDf["CaseID"].isin(validFileDf["CaseID"].unique().tolist())]
totalScopeFilterByFiles = len(targetCaseDf)

targetCaseDf = targetCaseDf[targetCaseDf["CaseID"].isin(filterPartDf["CaseID"].unique().tolist())]
totalScopeFilterByFilesAndPart = len(targetCaseDf)
targetCaseDf.drop_duplicates(subset=["CaseID"], inplace=True)
targetCaseDf.sort_values(by="CaseID", inplace=True)


In [42]:
targetCaseDf = targetCaseDf[targetCaseDf["Vehicle_Class"] == 2]
targetCaseDf = targetCaseDf[~targetCaseDf["Circumstances_of_Accident"].isin(circumstancesToRemove)]
targetCaseDf = targetCaseDf[targetCaseDf["Gross_Offer_Insurer"] > 0]
targetCaseDf = targetCaseDf[targetCaseDf["Claim_Type"].str.contains("OD|TP")]



In [43]:
filterCaseDf = pd.json_normalize([
    {
    "filter" : "remove_by_claim_type",
    "val" : removeByClaimType,
    },
    {
    "filter" : "remove_by_vehicle_class",
    "val" : removedByVehicleClass,
    },
    {
    "filter" : "remove_by_circumstances",
    "val" : removedByCircumstances,
    },
    {
    "filter" : "remove_by_gross_offer_insurer",
    "val" : removedByGrossOfferInsurer,
    },
    {
    "filter" : "remove_by_incomplete_images",
    "val" : removedByFiles,
    },
       {
    "filter" : "remove_by_empty_partlist",
    "val" : removedByParts,
    },

])


In [44]:
import plotly.express as ex


In [45]:
ex.bar(filterCaseDf, x="filter", y="val", text_auto=True)

In [46]:
totalRemoved = totalScopeFilterByFilesAndPart - len(targetCaseDf) 
totalRemoved

82840

In [47]:
totalRemoved = totalScopeFilterByFilesAndPart - len(targetCaseDf) 
totalRemoved


fig = go.Figure(data=[go.Pie(labels=["Case Data Used", 
                                     "Case Data Removed",
                                     ], \
                             values=[len(targetCaseDf), totalRemoved])])
fig.update_layout(title="Total case after remove by image files and partlist")

fig.show()

In [48]:
totalRemoved = totalScopeFilterByFiles - len(targetCaseDf) 
totalRemoved


fig = go.Figure(data=[go.Pie(labels=["Case Data Used", 
                                     "Case Data Removed",
                                     ], \
                             values=[len(targetCaseDf), totalRemoved])])
fig.update_layout(title="Total case after remove by image files only")

fig.show()

In [49]:
totalScope = len(caseDf) - removeByClaimType
totalRemoved = totalScope - len(targetCaseDf) 

fig = go.Figure(data=[go.Pie(labels=["Case Data Used", 
                                     "Case Data Removed",
                                     ], \
                             values=[len(targetCaseDf), totalRemoved])])
fig.update_layout(title="Total case after remove by claim type OD and TP")

fig.show()

In [50]:
totalScope = len(caseDf)
totalRemoved = totalScope - len(targetCaseDf) 

fig = go.Figure(data=[go.Pie(labels=["Case Data Used", 
                                     "Case Data Removed",
                                     ], \
                             values=[len(targetCaseDf), totalRemoved])])
fig.update_layout(title="Total case with no filter")


In [51]:
targetFileDesc = ["Front View Left", "Front View Right", "Rear View Left", "Rear View Right", "Front View", "Rear View"]

In [52]:
downloadFileDf = fileDf[(fileDf["StdDocDesc"].isin(targetFileDesc)) & (fileDf["CaseID"].isin(targetCaseDf["CaseID"].unique().tolist()))]
downloadFileDf

,CaseID,iDOCID,FinalDate,StdDocDesc
211094,10533333,464102036.0,2015-06-09 16:41:12.990000+00:00,Front View Left
742295,12458719,563578135.0,2017-01-03 08:03:17.917000+00:00,Front View Left
1285736,11050033,490004916.0,2017-03-14 14:47:14.893000+00:00,Front View Left
1002609,10923162,483540176.0,2017-05-19 10:43:27.907000+00:00,Front View Left
716887,10844645,479515095.0,2017-07-17 15:59:54.010000+00:00,Front View Left
...,...,...,...,...
84731,13447277,649096361.0,2022-11-05 16:19:43.353000+00:00,Rear View Left
138931,13555816,649181771.0,2022-11-06 19:49:31.097000+00:00,Rear View Left
136363,13548901,651304975.0,2022-11-15 10:50:56.400000+00:00,Rear View Left
605954,13313567,652815211.0,2022-11-22 15:01:45.557000+00:00,Rear View Left


In [53]:
targetCaseDf.drop_duplicates(subset="CaseID", inplace=True)
downloadFileDf = downloadFileDf.groupby(['CaseID', 'iDOCID']).first()
filterPartDf

,CaseID,Description,fVAL,DamageCond,CoType
0,12657878,bonnet,550.0,bent/dented,I
4,12657878,bumper front cover,350.0,creased/torn,I
6,12657878,bumper front lower grille,110.0,bracket torn,I
7,12657878,bumper reinforcement front,250.0,bent/dented,I
8,12657878,fog lamp front rh,150.0,bracket snapped,I
...,...,...,...,...,...
26637936,12908158,fuse / relay box,300.0,broken,I
26637939,12908158,airbag rotary coupling,500.0,activated,I
26637942,12908162,windscreen glass - front,309.2,,I
26637944,12908163,tint film,280.0,,I


In [54]:
frontViewLeftDf = wr.s3.read_parquet(f"s3://{bucket2}/",  dataset=True, partition_filter= lambda x: x["StdDocDesc"] == "Front View Left", columns=["CaseID", "iDOCID", "FinalDate", "StdDocDesc"])
frontViewRightDf = wr.s3.read_parquet(f"s3://{bucket2}/",  dataset=True, partition_filter= lambda x: x["StdDocDesc"] == "Front View Right", columns=["CaseID", "iDOCID", "FinalDate", "StdDocDesc"])
rearViewRightDf = wr.s3.read_parquet(f"s3://{bucket2}/",  dataset=True, partition_filter= lambda x: x["StdDocDesc"] == "Rear View Right", columns=["CaseID", "iDOCID", "FinalDate", "StdDocDesc"])
rearViewLeftDf = wr.s3.read_parquet(f"s3://{bucket2}/",  dataset=True, partition_filter= lambda x: x["StdDocDesc"] == "Rear View Left", columns=["CaseID", "iDOCID", "FinalDate", "StdDocDesc"])
rearViewDf = wr.s3.read_parquet(f"s3://{bucket2}/",  dataset=True, partition_filter= lambda x: x["StdDocDesc"] == "Rear View", columns=["CaseID", "iDOCID", "FinalDate", "StdDocDesc"])
frontViewDf = wr.s3.read_parquet(f"s3://{bucket2}/",  dataset=True, partition_filter= lambda x: x["StdDocDesc"] == "Front View", columns=["CaseID", "iDOCID", "FinalDate", "StdDocDesc"])

allViewFilesDf = [frontViewLeftDf, frontViewRightDf,  rearViewRightDf, rearViewLeftDf, frontViewDf, rearViewDf]

In [55]:
dfWithUniqueView = []
for dfId in range(len(allViewFilesDf)):
    df = allViewFilesDf[dfId]
    df["FinalDate"] = pd.to_datetime(df["FinalDate"])
    df.sort_values(by="FinalDate", inplace=True)
    df = df.groupby("CaseID").head(1)
    dfWithUniqueView.append(df)
completeFileDf = pd.concat(dfWithUniqueView)

In [56]:
completeFileDf = completeFileDf[completeFileDf["CaseID"].isin(targetCaseDf["CaseID"].unique().tolist())]

In [57]:
kwrgs = {
    "endpoint_url": "http://192.168.1.4:8333",
    "aws_access_key_id": "",
    "aws_secret_access_key": "",
    # "Username": "aaa",
}
scopeCaseBucket = "scope_case"
scopeFileBucket = "scope_file"
scopePartBucket = "scope_part"

cli = boto3.client("s3", **kwrgs)
# cli.create_bucket(Bucket=scopeCaseBucket)
# cli.create_bucket(Bucket=scopeFileBucket)
# cli.create_bucket(Bucket=scopePartBucket)

wr.s3.to_parquet(
                    df=targetCaseDf,
                    path=f"s3://{scopeCaseBucket}/",
                    dataset=True,
                    mode="overwrite",
                    partition_cols=["Vehicle_Type"],
                )
wr.s3.to_parquet(
                    df=completeFileDf,
                    path=f"s3://{scopeFileBucket}/",
                    dataset=True,
                    mode="overwrite",
                    partition_cols=["StdDocDesc"],
                )
wr.s3.to_parquet(
                    df=filterPartDf,
                    path=f"s3://{scopePartBucket}/",
                    dataset=True,
                    mode="overwrite",
                )

/home/alextay96/.local/lib/python3.10/site-packages/awswrangler/s3/_write_dataset.py:92: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.

/home/alextay96/.local/lib/python3.10/site-packages/awswrangler/s3/_write_dataset.py:92: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



{'paths': ['s3://scope_part/b53e7a7f81214e5fb966723353582bf1.snappy.parquet'],
 'partitions_values': {}}